In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

## Getting the Stations table

In [6]:
connection.rollback()

query = """

drop table if exists stations;

"""

cursor.execute(query)

connection.commit()


In [7]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

In [8]:
my_read_csv_file("../exercise/stations.csv", limit=10)

['station', 'latitude', 'longitude', 'transfer_time']
['12th Street', '37.803608', '-122.272006', '282']
['16th Street Mission', '37.764847', '-122.420042', '287']
['19th Street', '37.807869', '-122.26898', '67']
['24th Street Mission', '37.752', '-122.4187', '277']
['Antioch', '37.996281', '-121.783404', '0']
['Ashby', '37.853068', '-122.269957', '299']
['Balboa Park', '37.721667', '-122.4475', '48']
['Bay Fair', '37.697', '-122.1265', '63']
['Berryessa', '37.368361', '-121.874655', '288']

Printed  10 lines of  51 total lines.


In [9]:
connection.rollback()

query = """

copy stations
from '/user/projects/project-3-RobertMeyer289/exercise/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stations
order by station
limit 5

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0


## Getting the Lines table

In [11]:
connection.rollback()

query = """

drop table if exists lines;

"""

cursor.execute(query)

connection.commit()


In [12]:
connection.rollback()

query = """

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

"""

cursor.execute(query)

connection.commit()

In [13]:
my_read_csv_file("../exercise/lines.csv", limit=10)

['line', 'sequence', 'station']
['blue', '1', 'Dublin']
['blue', '2', 'West Dublin']
['blue', '3', 'Castro Valley']
['blue', '4', 'Bay Fair']
['blue', '5', 'San Leandro']
['blue', '6', 'Coliseum']
['blue', '7', 'Fruitvale']
['blue', '8', 'Lake Merritt']
['blue', '9', 'West Oakland']

Printed  10 lines of  115 total lines.


In [14]:
connection.rollback()

query = """

copy lines
from '/user/projects/project-3-RobertMeyer289/exercise/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from lines
order by line, sequence

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,sequence,station
0,blue,1,Dublin
1,blue,2,West Dublin
2,blue,3,Castro Valley
3,blue,4,Bay Fair
4,blue,5,San Leandro
...,...,...,...
109,yellow,23,Daly City
110,yellow,24,Colma
111,yellow,25,South San Francisco
112,yellow,26,San Bruno


## Getting the Travel Times table

In [16]:
connection.rollback()

query = """

drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()


In [17]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

"""

cursor.execute(query)

connection.commit()

In [18]:
my_read_csv_file("../exercise/travel_times.csv", limit=10)

['station_1', 'station_2', 'travel_time']
['12th Street', '19th Street', '120']
['12th Street', 'Lake Merritt', '180']
['12th Street', 'West Oakland', '300']
['16th Street Mission', '24th Street Mission', '120']
['16th Street Mission', 'Civic Center', '180']
['19th Street', 'MacArthur', '180']
['24th Street Mission', 'Glen Park', '180']
['Antioch', 'Pittsburg Center', '420']
['Ashby', 'Downtown Berkeley', '180']

Printed  10 lines of  52 total lines.


In [19]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/project-3-RobertMeyer289/exercise/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from travel_times
order by station_1, station_2
limit 5

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station_1,station_2,travel_time
0,12th Street,19th Street,120
1,12th Street,Lake Merritt,180
2,12th Street,West Oakland,300
3,16th Street Mission,24th Street Mission,120
4,16th Street Mission,Civic Center,180
